# Interpolate

interpolate new data into well header

In [1]:
# ONLY RUN ONCE, must restart notebook otherwise
import pandas as pd
import os
from pathlib import Path
current_dir = Path.cwd()
path = current_dir.parent
print(path)
#change directory to get src class data
os.chdir(path)
from src.data_object import *
from src.transforms import *

from src.calculable_object import *
from src.wellbore_trajectory import *


from scipy import interpolate

C:\Users\BpAmos\Documents\repos\directional-survey-converter


## Get Files

In [2]:
file_paths = get_files(path, folders='data')
file_paths.items

[WindowsPath('C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/wellbore_survey.csv'),
 WindowsPath('C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/wellbore_survey.json'),
 WindowsPath('C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/wellbore_survey_many.csv'),
 WindowsPath('C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/well_C.csv'),
 WindowsPath('C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/well_header.csv'),
 WindowsPath('C:/Users/BpAmos/Documents/repos/directional-survey-converter/data/well_interpolate_data.csv')]

In [3]:
filePath = file_paths[0]
filePath
df_well = pd.read_csv(filePath, sep=',')
df_well = df_well[['wellId','md','inc','azim']]
df_well.head()

,wellId,md,inc,azim
0,well_C,0.00,0.00,227.11
1,well_C,35.00,0.00,227.11
2,well_C,774.81,0.46,227.11
3,well_C,800.00,0.13,163.86
4,well_C,900.00,0.57,230.43


In [5]:
filePath = file_paths[5]
df_interp = pd.read_csv(filePath, sep=',')
df_interp.head()

,wellId,var_type,name,top_depth,bottom_depth
0,well_C,wellTop,top_A,1234,1450
1,well_C,wellTop,top_B,1450,1625


In [6]:
filePath = file_paths[3]
df_interp = pd.read_csv(filePath, sep=',')
df_interp.head()

,wellId,var_type,name,top_depth,bottom_depth
0,well_C,wellTop,top_A,1234,1450
1,well_C,wellTop,top_B,1450,1625


In [179]:
interp_dict = df_interp.to_dict('records')
wellId = interp_dict[0]['wellId']
var_type = interp_dict[0]['var_type']
name = interp_dict[0]['name']
name

'top_A'

In [178]:
interp_vals = np.array([interp_dict[0]['top_depth'],interp_dict[0]['bottom_depth']])
interp_vals

array([1234, 1450])

In [185]:


new_dict = {'wellId':wellId,
           "var_type":var_type,
            "type":{'name':name, 'values':interp_vals}}
new_dict

{'wellId': 'well_C',
 'var_type': 'wellTop',
 'type': {'name': 'top_A', 'values': array([1234, 1450])}}

In [152]:
md = np.array(df_well['md'])
inc = np.array(df_well['inc'])
azim = np.array(df_well['azim'])
#print(md)
#print(inc)
index_val = np.array(df_well.index, float)
#index_val

In [61]:
new_md = np.array(df_interp['top_depth'])
new_md

array([1234, 1450], dtype=int64)

In [243]:
def interp_new_values(interp_md, md, y):
    
    
    # stack original arrays
    original_vals =  np.stack((md, y))

    
    # interp new param values from md and provided values
    y_new = np.interp(interp_md,md,y)

    # stack the new md values and the new interp values
    interp_vals = np.stack((interp_md, y_new))

    
    # concat original array stack and new interp value array stack
    concat_arrays = np.concatenate((original_vals,interp_vals),axis=1)
    
    # sort the new concat arrays by md
    sorted_arr = concat_arrays[:, concat_arrays[0].argsort()]
    
    md, y = sorted_arr[0], sorted_arr[1]
    
    return md,y    

In [271]:
def interp_vals(interp_md, md, inc_arr, azim_arr):
    
    md_new,inc_arr = interp_new_values(interp_md, md, inc_arr)
    #print(md)
    md_new,azim_arr = interp_new_values(interp_md, md, azim_arr)
    #print(md)
    
    interp_vals = np.stack((md_new, inc_arr,azim_arr),axis=1)
    #print(interp_vals)
    
    df_array = pd.DataFrame(data=interp_vals, columns=["md", "inc", 'azim'])
    
    return df_array

In [283]:
md = np.array(df_well['md'])
inc = np.array(df_well['inc'])
azim = np.array(df_well['azim'])

interp_md = np.array(df_interp['top_depth'])
y = azim

#md,y = interp_new_values(interp_md, md, y)
# print(md)
# print(y)

df_array = interp_vals(interp_md, md, inc, azim)
df_array.head(20)


# convert to series
s = pd.Series(df_array['md'])
s = s.between(interp_md.min(), interp_md.max(), inclusive=True)

# replace column
df_array['well_top'] = s
df_array['well_top'] = np.where(df_array['well_top']==True, name, None)
df_array.head(20)

,md,inc,azim,well_top
0,0.00,0.0000,227.1100,None
1,35.00,0.0000,227.1100,None
2,774.81,0.4600,227.1100,None
3,800.00,0.1300,163.8600,None
4,900.00,0.5700,230.4300,None
5,1000.00,0.4900,254.4700,None
6,1100.00,0.2900,268.2500,None
7,1200.00,0.6000,147.6700,None
8,1234.00,0.5762,177.8926,top_A
9,1300.00,0.5300,236.5600,top_A


In [238]:
df_array = pd.DataFrame(data=numpy_data, index=["row1", "row2"], columns=["column1", "column2"])

array([    0.  ,    35.  ,   774.81,   800.  ,   900.  ,  1000.  ,
        1100.  ,  1200.  ,  1234.  ,  1300.  ,  1400.  ,  1450.  ,
        1500.  ,  1600.  ,  1700.  ,  1800.  ,  1900.  ,  2000.  ,
        2100.  ,  2200.  ,  2300.  ,  2400.  ,  2500.  ,  2600.  ,
        2700.  ,  2800.  ,  2900.  ,  3000.  ,  3100.  ,  3200.  ,
        3300.  ,  3400.  ,  3500.  ,  3600.  ,  3700.  ,  3800.  ,
        3900.  ,  4000.  ,  4100.  ,  4200.  ,  4300.  ,  4400.  ,
        4500.  ,  4600.  ,  4700.  ,  4800.  ,  4900.  ,  5000.  ,
        5100.  ,  5200.  ,  5300.  ,  5400.  ,  5500.  ,  5600.  ,
        5700.  ,  5800.  ,  5900.  ,  6000.  ,  6100.  ,  6200.  ,
        6300.  ,  6400.  ,  6450.67,  6532.  ,  6625.  ,  6720.  ,
        6813.  ,  6909.  ,  7002.  ,  7098.  ,  7191.  ,  7286.  ,
        7379.  ,  7475.  ,  7569.  ,  7663.  ,  7758.  ,  7851.  ,
        7947.  ,  8040.  ,  8135.  ,  8229.  ,  8324.  ,  8418.  ,
        8512.  ,  8606.  ,  8700.  ,  8794.  ,  8859.  ,  8924

In [217]:
new_array = np.array(['yes','yes'])
new_array2 = np.array([3,4])
new_array

array(['yes', 'yes'], dtype='<U3')

In [219]:
np.stack((new_array,new_array2,new_array))

array([['yes', 'yes'],
       ['3', '4'],
       ['yes', 'yes']], dtype='<U11')

In [229]:
md.size
a = np.empty(md.size, dtype=object) 
a

array([None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None],
      dtype=object)

In [9]:
# use that path to greate a Deviation Survey Object
my_calc_obj = WellboreTrajectory(data)
my_calc_obj.deviation_survey_obj

DeviationSurvey(wellId='well_C', md=array([    0.  ,    35.  ,   774.81,   800.  ,   900.  ,  1000.  ,
        1100.  ,  1200.  ,  1300.  ,  1400.  ,  1500.  ,  1600.  ,
        1700.  ,  1800.  ,  1900.  ,  2000.  ,  2100.  ,  2200.  ,
        2300.  ,  2400.  ,  2500.  ,  2600.  ,  2700.  ,  2800.  ,
        2900.  ,  3000.  ,  3100.  ,  3200.  ,  3300.  ,  3400.  ,
        3500.  ,  3600.  ,  3700.  ,  3800.  ,  3900.  ,  4000.  ,
        4100.  ,  4200.  ,  4300.  ,  4400.  ,  4500.  ,  4600.  ,
        4700.  ,  4800.  ,  4900.  ,  5000.  ,  5100.  ,  5200.  ,
        5300.  ,  5400.  ,  5500.  ,  5600.  ,  5700.  ,  5800.  ,
        5900.  ,  6000.  ,  6100.  ,  6200.  ,  6300.  ,  6400.  ,
        6450.67,  6532.  ,  6625.  ,  6720.  ,  6813.  ,  6909.  ,
        7002.  ,  7098.  ,  7191.  ,  7286.  ,  7379.  ,  7475.  ,
        7569.  ,  7663.  ,  7758.  ,  7851.  ,  7947.  ,  8040.  ,
        8135.  ,  8229.  ,  8324.  ,  8418.  ,  8512.  ,  8606.  ,
        8700.  ,  8794.  ,

In [33]:
my_calc_obj.calculate_survey_points()
my_calc_obj.deviation_survey_obj

DeviationSurvey(wellId='well_C', md=array([    0.  ,    35.  ,   774.81,   800.  ,   900.  ,  1000.  ,
        1100.  ,  1200.  ,  1300.  ,  1400.  ,  1500.  ,  1600.  ,
        1700.  ,  1800.  ,  1900.  ,  2000.  ,  2100.  ,  2200.  ,
        2300.  ,  2400.  ,  2500.  ,  2600.  ,  2700.  ,  2800.  ,
        2900.  ,  3000.  ,  3100.  ,  3200.  ,  3300.  ,  3400.  ,
        3500.  ,  3600.  ,  3700.  ,  3800.  ,  3900.  ,  4000.  ,
        4100.  ,  4200.  ,  4300.  ,  4400.  ,  4500.  ,  4600.  ,
        4700.  ,  4800.  ,  4900.  ,  5000.  ,  5100.  ,  5200.  ,
        5300.  ,  5400.  ,  5500.  ,  5600.  ,  5700.  ,  5800.  ,
        5900.  ,  6000.  ,  6100.  ,  6200.  ,  6300.  ,  6400.  ,
        6450.67,  6532.  ,  6625.  ,  6720.  ,  6813.  ,  6909.  ,
        7002.  ,  7098.  ,  7191.  ,  7286.  ,  7379.  ,  7475.  ,
        7569.  ,  7663.  ,  7758.  ,  7851.  ,  7947.  ,  8040.  ,
        8135.  ,  8229.  ,  8324.  ,  8418.  ,  8512.  ,  8606.  ,
        8700.  ,  8794.  ,

In [ ]:
@classmethod
def from_df(cls,  df:DataFrame, wellId_name: Optional[str] = None, md_name: Optional[str] = None,
            inc_name: Optional[str] = None, azim_name: Optional[str] = None,
            surface_latitude_name: Optional[str] = None,
            surface_longitude_name: Optional[str] = None) -> 'DirectionalSurveyObj':

    # if no column names are specified use 0:5 for dir survey obj
    wellId_name = ifnone(wellId_name, df.columns[0])
    md_name = ifnone(md_name, df.columns[1])
    inc_name = ifnone(inc_name, df.columns[2])
    azim_name = ifnone(azim_name, df.columns[3])
    surface_latitude_name = ifnone(surface_latitude_name, df.columns[4])
    surface_longitude_name = ifnone(surface_longitude_name, df.columns[5])

    dataclass_obj = dict(wellId=df[wellId_name][0],
                               md=np.array(df[md_name]),
                               inc=np.array(df[inc_name]),
                               azim=np.array(df[azim_name]),
                               surface_latitude=df[surface_latitude_name][0],
                               surface_longitude=df[surface_longitude_name][0])
    return dataclass_obj

def from_csv(self, **kwargs):
    if self.extension == '.csv':
        print(self.path)
        # go through csv steps
        # Transform.from_csv: > return self.from_df
        # self.from_df > return self.from_json
        df = pd.read_csv(self.path, sep=',')
        return self.from_df(df, **kwargs)

In [286]:
nums = [1,2,3,4]

counter = 0
res = []
for i in nums:
    counter += i
    res.append(counter)
myList

[1, 3, 6, 10]

In [292]:
nums = [1,2,3,4]

[sum(nums[:i+1]) for i in range(len(nums))]


[1, 3, 6, 10]